In [73]:
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from copy import deepcopy

from model import Model
import utils
from early_stopping import EarlyStopping
from priors import moving_average

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
config = utils.read_config()
# set seeds
utils.set_seeds(config['seed'])
# utils.set_seeds(7)

In [76]:
behaviour_data, spikes = utils.load_dataset(config)
# consider data from only t = -1
# time_from = int(1/bin_len)
# behaviour_data, spikes = [x[time_from:, :] for x in behaviour_data], [x[time_from:, :] for x in spikes]
num_trials, time_bins, emissions_dim = np.array(spikes).shape

In [77]:
stim_idx, choice_idx = 6, 3
stim = [x[0, stim_idx] for x in behaviour_data]
choice = [x[0, choice_idx] for x in behaviour_data]
num_contacts = [np.sum(x[:, -9:-5], axis=1) for x in behaviour_data]
# concat them
behaviour_data = np.stack((stim, choice), axis=1)

In [78]:
# convert to torch tensors
behaviour_data = torch.tensor(behaviour_data, dtype=torch.long)
# behaviour_data = torch.tensor(behaviour_data, dtype=torch.float32)
spikes = torch.tensor(spikes, dtype=torch.float32)

In [79]:
# create dataloader with random sampling for training and testing
# split data into training and testing
behaviour_data_train, behaviour_data_test, spikes_train, spikes_test = train_test_split(behaviour_data, spikes, test_size=0.3, random_state=42)
# behaviour_data_train, behaviour_data_test, spikes_train, spikes_test = train_test_split(behaviour_data, spikes, test_size=0.2, random_state=7)

# create dataloaders
train_dataset = TensorDataset(behaviour_data_train, spikes_train)
test_dataset = TensorDataset(behaviour_data_test, spikes_test)

batch_size = config['batch_size']
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [80]:
# distribution of choice and stimulus in test
print("Train distribution of Stimulus: {}, Choice: {}".format(np.mean(behaviour_data_train[:, 0].numpy()), np.mean(behaviour_data_train[:, 1].numpy())))
print("Test distribution of Stimulus: {}, Choice: {}".format(np.mean(behaviour_data_test[:, 0].numpy()), np.mean(behaviour_data_test[:, 1].numpy())))

Train distribution of Stimulus: 0.5089285714285714, Choice: 0.4375
Test distribution of Stimulus: 0.42857142857142855, Choice: 0.2653061224489796


In [81]:
# mean firing rate of neurons in tran spikes
neuron_bias = torch.mean(spikes_train, dim=0)

In [82]:
# # check if mps is available
# device = torch.device('mps' if torch.backends.mps.is_built() else 'cpu')
# print(device)
# model = model.to(device)
# spikes = spikes.to(device)

In [83]:
def test(model, test_loader, n_samples=200):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for _, (behavior_batch, spikes_batch) in enumerate(test_loader):
            y_recon, mu, A, z, x, behavior = model.forward(spikes_batch, n_samples=n_samples)
            _, loss_l = model.loss(None, spikes_batch, y_recon, mu, A, z, x, behavior, behavior_batch)
            # l.append(loss_l[1])
            test_loss += np.array(loss_l)            
            # print(np.mean(l), np.std(l))
    # divide loss by total number of samples in dataloader    
    return test_loss/len(test_loader)

In [85]:
config = utils.read_config()
# training loop
num_epochs = config['epochs']
# create model and optimizer
model = Model(config, input_dim=emissions_dim) #, neuron_bias=neuron_bias
# model = torch.compile(model)
early_stop = EarlyStopping(patience=config['early_stop']['patience'], delta=config['early_stop']['delta'], trace_func=print)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', threshold=1, verbose=True, patience=5, factor=0.5)
# print named parameters of model
# print("Model's state_dict:")
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data.shape)

Not using moving average
Scheduler not implemented for GRU
Number of trainable parameters in VAE: 5888
No behavior decoder


In [86]:
torch.autograd.set_detect_anomaly(True)
train_losses, test_losses = [], []

def train(model: Model, train_loader, val_loader):    
    test_every = config['test_every']    
    train_decoder_after = config['decoder']['train_decoder_after']    
    num_samples_train = config['num_samples_train']
    save_model = True    
    for epoch in range(num_epochs):
        # forward pass
        # print(model.behavior_decoder.scheduler.get_last_lr())
        # model.vae.scheduler.get_last_lr()
        epoch_loss = 0
        model.train()
        for i, (behavior_batch, spikes_batch) in enumerate(train_loader):            
            # behavior_batch = behavior_batch.long()
            y_recon, mu, A, z, x, behavior_pred = model(spikes_batch, n_samples=num_samples_train)
            # calculate loss
            loss, loss_l = model.loss(epoch, spikes_batch, y_recon, mu, A, z, x, behavior_pred, behavior_batch)            
            # backward pass
            model.optim_zero_grad()
            loss.backward()
            # print gradient of any weight
            # if epoch > 10:
            #     print(model.behavior_decoder.conv_choice[1].weight.grad)            
            model.optim_step(train_decoder = epoch >= train_decoder_after)                
            epoch_loss += np.array(loss_l)
        
        # if epoch % 100 == 0:
        #     # print lr of decoder
        #     print(model.behavior_decoder.scheduler.get_last_lr())
        train_losses.append((epoch, epoch_loss/len(train_loader)))
        model.scheduler_step(step_decoder = epoch >= train_decoder_after)
        # test loss
        if (epoch+1) % test_every == 0:            
            test_loss = test(model, val_loader)
            sum_test_loss = np.sum(test_loss)
            # scheduler.step(sum_test_loss)
            test_losses.append((epoch, test_loss))
            early_stop(sum_test_loss, model, save_model=save_model, save_prefix='best')
            model.save_model(save_prefix=str(epoch))
            print('Epoch [{}/{}], Train Loss: {}, Test Loss: {}, Best Loss: {}'.format(epoch+1, num_epochs, train_losses[-1][1], test_losses[-1][1], early_stop.best_score))
            if early_stop.slow_down:
                test_every = config['early_stop']['test_every_new']
            else:
                test_every = config['test_every']
            if early_stop.early_stop:
                print("Early stopping")
                break
            
    
    only_test_loss = [np.sum(x[1]) for x in test_losses]
    
    # compute min test loss and return it    
    # return np.min(only_test_loss), train_losses, test_losses
    
    # compute median of test loss in a window of 5
    meds = []
    half_window = 10
    only_test_loss = [0]*(half_window) + only_test_loss + [0]*(half_window)
    for i in range(half_window, len(only_test_loss)-half_window):
        meds.append(np.max(only_test_loss[i-half_window:i+half_window]))
    return np.min(meds), train_losses, test_losses

_ = train(model, train_loader, test_loader)
# train model
# min_test_loss, train_losses, test_losses = train(model, test_loader)

Epoch [20/2000], Train Loss: [570.05297852], Test Loss: [587.59106445], Best Loss: 587.591064453125


In [ ]:
train_losses_og, test_losses_og = train_losses[:], test_losses[:]

In [ ]:
# model.prior_modules[0].plot_gaussian()
# model.prior_modules[0].log_std

In [ ]:
utils.plot_curve(model, config, train_losses, test_losses)

In [ ]:
# sort list by test loss
sorted_loss = sorted(test_losses, key=lambda x: x[1][1], reverse=True) 
# extract first element after epoch 600
sorted_loss = [(x[0], x[1][1]/100) for x in sorted_loss if x[0] > 100]
print(sorted_loss[:2], sorted_loss[-10:])
# print(test_losses)

In [ ]:
# load best model
model.load_model('best')
# load model from epoch x
# model.load_model('159')

In [ ]:
# lin_maps = model.vae.linear_maps
# # c1, c2 = lin_maps[0].weight.detach().numpy(), lin_maps[1].weight.detach().numpy()
# # print(c1.T.dot(c2)/(np.linalg.norm(c1)*np.linalg.norm(c2)))
# c1, c2, c3 = lin_maps[0].weight.detach().numpy(), lin_maps[1].weight.detach().numpy(), lin_maps[2].weight.detach().numpy()
# print("Norms: {}, {}, {}".format(np.linalg.norm(c1), np.linalg.norm(c2), np.linalg.norm(c3)))

In [ ]:
with torch.no_grad():
    model.eval()
    y_recon, mu, A, _, _, _ = model.forward(spikes, n_samples=1)
    # run on only test
    y_recon_test, mu_test, A_test, _, _, _ = model.forward(spikes_test, n_samples=1)
    
# convert to numpy
y_recon_np = y_recon.detach().numpy()
spikes_np = spikes.detach().numpy()
y_recon_test_np = y_recon_test.detach().numpy()
spikes_test_np = spikes_test.detach().numpy()
# compute bits/spike
bits_per_spike_all = utils.bits_per_spike(y_recon_np, spikes_np)
bits_per_spike_test = utils.bits_per_spike(y_recon_test_np, spikes_test_np)
# show distribution of bits per spike
plt.hist(bits_per_spike_all, bins=50)
plt.xlabel('Bits/spike')
plt.ylabel('Frequency')
plt.show()
# print('Bits per spike: {}'.format(bits_per_spike))
print("Bits per spike all: {}, test: {}".format(np.sum(bits_per_spike_all), np.sum(bits_per_spike_test)))

In [ ]:
a_t = np.mean(A.numpy()[0:1, 20, :, :], axis=0)
cov = a_t * a_t.T
# print(cov.shape, spikes_np.shape)
plt.imshow(cov)
plt.colorbar()

In [ ]:
# plot PSTH of reconstructed and original data
averaged_recon, averaged_original = y_recon.mean(axis=0), spikes_np.mean(axis=0)
# stimulus and choice important
common = [12, 14, 4, 31]
stim_neurons = [15, 11, 33, 30]
choice_neurons = [16, 2, 6, 8]
# plot each in a 5x7 grid
fig, axs = plt.subplots(5, 7, figsize=(12, 9))
# set title of figure
fig.suptitle('yellow: choice, green: stimulus, pink: common')
for i in range(5):
    for j in range(7):
        neuron_idx = i*7+j        
        axs[i, j].plot(averaged_recon[:, neuron_idx], label='recon', color='red')
        axs[i, j].plot(averaged_original[:, neuron_idx], label='original', color='blue')
        # no ticks
        axs[i, j].set_xticks([])
        axs[i, j].set_yticks([])
        # set title of plot to neuron index
        # print only 2 decimal places        
        axs[i, j].set_title('{}: {:.4f}'.format(neuron_idx, bits_per_spike_all[neuron_idx]))
        # set background color of plot to green if neuron in choice
        if neuron_idx in choice_neurons:
            axs[i, j].set_facecolor('yellow')
        # set background color of plot to red if neuron in stimulus
        if neuron_idx in stim_neurons:
            axs[i, j].set_facecolor('green')
        # set background color of plot to blue if neuron in common
        if neuron_idx in common:
            axs[i, j].set_facecolor('pink')
axs[0, 0].legend()

In [ ]:
# # plot trial averaged latent space
# with torch.no_grad():
#     _, (mu, A), (z, x), _ = model.forward(spikes)
#     z, x = z.numpy(), x.numpy()
# # z, x = torch.nn.Softmax(dim=2)(mu[:, :, :model.vae.z_dim]).numpy(), mu[:, :, model.vae.z_dim:].numpy()
# # z_std, x_std = np.std(z, axis=0), np.std(x, axis=0)
# fig, axs = plt.subplots(2, 1, figsize=(4, 8))
# z_avg, x_avg = np.mean(z, axis=0), np.mean(x, axis=0)
# # make x ticks of range 0.1 from -2 to 0.5
# bin_len = config['shape_dataset']['win_len']
# t = np.arange(-2, 0.5, bin_len)
# for i in range(z.shape[2]):
#     axs[0].plot(t, z_avg[:, i], label='z{}'.format(i))
#     axs[1].plot(t, x_avg[:, i], label='x{}'.format(i))
# axs[0].set_title('z')
# axs[1].set_title('x')
# axs[0].legend()
# axs[1].legend()

# plot trial averaged latent space
z, x = torch.nn.Softmax(dim=2)(mu[:, :, :model.vae.z_dim]).numpy(), mu[:, :, model.vae.z_dim:].numpy()
z_std, x_std = np.std(z, axis=0), np.std(x, axis=0)
z_avg, x_avg = np.mean(z, axis=0), np.mean(x, axis=0)
# make x ticks of range 0.1 from -2 to 0.5
bin_len = config['shape_dataset']['win_len']
t = np.arange(-2, 0.5, bin_len)
for i in range(z.shape[2]):
    plt.plot(t, z_avg[:, i], label='z{}'.format(i))    
    plt.fill_between(t, z_avg[:, i]-z_std[:, i], z_avg[:, i]+z_std[:, i], alpha=0.3)
# plt.set_title('z')
plt.legend()

In [ ]:
print(model.arch_name)

In [ ]:
def plot_cont_latent(data, label, ax_id, linestyle):
    num_latents = data.shape[1]
    for i in range(num_latents):
        axs[ax_id].plot(t, data[:, i], label='{}_x{}'.format(label, i),
                        color=colors[i], linestyle=linestyle)    
        
colors = ['red', 'blue', 'green', 'yellow', 'black', 'pink']
# group x for stimulus and choice
stim, choice = behaviour_data[:, 0].numpy(), behaviour_data[:, 1].numpy()
# group x for stimulus
x_stim_left, x_stim_right = x[stim == 1].mean(axis=0), x[stim == 0].mean(axis=0)
x_choice_left, x_choice_right = x[choice == 1].mean(axis=0), x[choice == 0].mean(axis=0)
# plot x for stimulus and choice
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

plot_cont_latent(x_stim_left, 'stim left', 0, '-')
plot_cont_latent(x_stim_right, 'stim right', 0, '-.')
axs[0].set_title('x grouped by stimulus')
axs[0].legend()

plot_cont_latent(x_choice_left, 'choice left', 1, '-')
plot_cont_latent(x_choice_right, 'choice right', 1, '-.')
axs[1].set_title('x grouped by choice')
axs[1].legend()


In [ ]:
# plot x on 2d
choice = behaviour_data[:, 1].numpy()
# group x for stimulus
x_choice_left, x_choice_right = x[choice == 1].mean(axis=0), x[choice == 0].mean(axis=0)
z_choice_left, z_choice_right = z[choice == 1].mean(axis=0), z[choice == 0].mean(axis=0)
# plot first 2 dimensions
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(x_choice_left[:, 2], x_choice_left[:, 3], label='choice left', color='red')
ax.plot(x_choice_right[:, 2], x_choice_right[:, 3], label='choice right', color='blue')
ax.set_title('x grouped by choice')

In [ ]:
import decoder
with torch.no_grad():
    model.eval()    
    _, (mu_train, A_train), (z_train, x_train), behavior_pred_train = model.forward(spikes_train)
    _, (mu_test, A_test), (z_test, x_test), behavior_pred_test = model.forward(spikes_test)
# if behavior_pred_train is None:
# train the linear decoder for behavior
# create linear decoder
linear_decoder = decoder.CNNDecoder(config, input_dim=2)
optimizer = torch.optim.Adam(linear_decoder.parameters(), lr=0.01)
decoder_train_l, decoder_test_l = [], []
for epoch in range(100):
    linear_decoder.train()
    # forward pass        
    linear_decoder.train()
    behavior_pred = linear_decoder(x_train, z_train)
    # behavior_pred = linear_decoder(mu_train[:, :, :2], mu_train[:, :, 2:])
    loss = linear_decoder.loss(behavior_pred, behaviour_data_train, None)
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()    
    epoch_loss = loss.item()    
    decoder_train_l.append(epoch_loss)
    # test loss
    if (epoch+1) % 1 == 0:
        linear_decoder.eval()
        test_pred = linear_decoder(x_test, z_test)
        # behavior_pred = linear_decoder(mu_train[:, :, :2], mu_train[:, :, 2:])
        test_loss = linear_decoder.loss(test_pred, behaviour_data_test, None).item()        
        decoder_test_l.append(test_loss)
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch+1, num_epochs, decoder_train_l[-1], decoder_test_l[-1]))
    # if epoch % 10 == 0:
    #     torch.save(model.state_dict(), os.path.join(base_path, 'vae_model_{}.pt'.format(epoch)))
    #     print('Model saved at epoch {}'.format(epoch))
with torch.no_grad():
    behavior_pred_train = linear_decoder(x_train, z_train).detach()
    behavior_pred_test = linear_decoder(x_test, z_test).detach()
    # behavior_pred_train = linear_decoder(mu_train[:, :, :2], mu_train[:, :, 2:])
    # behavior_pred_test = linear_decoder(mu_test[:, :, :2], mu_test[:, :, 2:])

In [ ]:
num_samples = 100
agg_pred_train, agg_pred_test = [], []
agg_y_train, agg_y_test = [], []

# convert to numpy
y_train = behaviour_data_train.numpy()
y_test = behaviour_data_test.numpy()
# accuracy of stimulus and choice
acc_stim_train, acc_stim_test = [], []
acc_choice_train, acc_choice_test = [], []

with torch.no_grad():
    model.eval()
    _, _, _, _, _, behavior_pred_train_all = model.forward(spikes_train, n_samples=num_samples)
    _, _, _, _, _, behavior_pred_test_all = model.forward(spikes_test, n_samples=num_samples)        
    for k in range(num_samples):
        b1 = behavior_pred_train_all.shape[0]//num_samples
        b2 = behavior_pred_test_all.shape[0]//num_samples
        behavior_pred_train = behavior_pred_train_all[k*b1: k*b1+b1]
        behavior_pred_test = behavior_pred_test_all[k*b2: k*b2+b2]
        # print(behavior_pred_test.shape, y_test.shape)
        # convert to numpy
        # print(behavior_pred_train[20:30, 1])
        # pred_train = behavior_pred_train.numpy() > 0
        # pred_test = behavior_pred_test.numpy() > 0        
        pred_train_stim = torch.argmax(behavior_pred_train[:, :2], dim=1).numpy()
        pred_test_stim = torch.argmax(behavior_pred_test[:, :2], dim=1).numpy()
        pred_train_choice = torch.argmax(behavior_pred_train[:, 2:], dim=1).numpy()
        pred_test_choice = torch.argmax(behavior_pred_test[:, 2:], dim=1).numpy()
        pred_train = np.concatenate((pred_train_stim[:, None], pred_train_choice[:, None]), axis=1)
        pred_test = np.concatenate((pred_test_stim[:, None], pred_test_choice[:, None]), axis=1)        
        # pred_test = behavior_pred_test.numpy() > 0
        # compute accuracy        
        accuracy_train_stim = accuracy_score(y_train[:, 0], pred_train[:, 0])
        accuracy_test_stim = accuracy_score(y_test[:, 0], pred_test[:, 0])        
        # do the same for choice
        accuracy_train_choice = accuracy_score(y_train[:, 1], pred_train[:, 1])
        accuracy_test_choice = accuracy_score(y_test[:, 1], pred_test[:, 1])
        # append to list
        acc_stim_train.append(accuracy_train_stim)
        acc_stim_test.append(accuracy_test_stim)
        acc_choice_train.append(accuracy_train_choice)
        acc_choice_test.append(accuracy_test_choice)
# print accuracy
print('Stimulus Accuracy - train: {:.4f} +- {:.4f}, test: {:.4f} +- {:.4f}'.format(np.mean(acc_stim_train), np.std(acc_stim_train), np.mean(acc_stim_test), np.std(acc_stim_test)))
print('Choice Accuracy - train: {:.4f} +- {:.4f}, test: {:.4f} +- {:.4f}'.format(np.mean(acc_choice_train), np.std(acc_choice_train), np.mean(acc_choice_test), np.std(acc_choice_test)))

In [ ]:
# examine a random trial
trial_idx = np.random.randint(num_trials)
# trial_idx = 20
# plot z and x
fig, axs = plt.subplots(3, 1, figsize=(5, 8))
# plot z
bin_len = config['shape_dataset']['win_len']
t = np.arange(-2, 0.5, bin_len)
axs[0].plot(t, z[trial_idx, :, 0], label='z0')
axs[0].plot(t, z[trial_idx, :, 1], label='z1')
# axs[0].plot(t, z[trial_idx, :, 2], label='z2')
axs[0].set_title('Mean of z for a random trial')
axs[0].set_ylim(0, 1)
axs[0].legend()
axs[0].set_xticks([])
# plot num contacts
axs[1].plot(t, num_contacts[trial_idx])
axs[1].set_title('num contacts')
axs[1].set_xticks([])
# plot x
axs[2].plot(t, x[trial_idx, :, 0], label='x0')
axs[2].plot(t, x[trial_idx, :, 1], label='x1')
# axs[2].plot(t, x[trial_idx, :, 2], label='x2')
axs[2].set_title('x, stimulus: {}, choice: {}'.format(stim[trial_idx].astype(int), choice[trial_idx].astype(int)))
axs[2].set_ylim(-2, 2)
axs[2].legend()

In [ ]:
# examine a random trial
# trial_idx = np.random.randint(len(spikes_test))
trial_idx = 20
# plot z and x
fig, axs = plt.subplots(3, 1, figsize=(5, 8))
# plot z
bin_len = config['shape_dataset']['win_len']
t = np.arange(-2, 0.5, bin_len)
with torch.no_grad():
    model.eval()
    _, (mu_train, A_train), (z_sample, x_sample), behavior_pred_train = model.forward(spikes[trial_idx: trial_idx+1])
z_sample, x_sample = z_sample[0].numpy(), x_sample[0].numpy()
axs[0].plot(t, z_sample[:, 0], label='z0')
axs[0].plot(t, z_sample[:, 1], label='z1')
# axs[0].plot(t, z[trial_idx, :, 2], label='z2')
axs[0].set_title('Sample of z for a random trial')
axs[0].set_ylim(0, 1)
axs[0].legend()
axs[0].set_xticks([])
# plot num contacts
axs[1].plot(t, num_contacts[trial_idx])
axs[1].set_title('num contacts')
axs[1].set_xticks([])
# plot x
axs[2].plot(t, x_sample[:, 0], label='x0')
axs[2].plot(t, x_sample[:, 1], label='x1')
# axs[2].plot(t, x[trial_idx, :, 2], label='x2')
axs[2].set_title('x, stimulus: {}, choice: {}'.format(stim[trial_idx].astype(int), choice[trial_idx].astype(int)))
axs[2].set_ylim(-2, 2)
axs[2].legend()

In [ ]:
# train a sklearn logistic regression model for comparison
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

def reshape(x, y):
    trials, time, dim = x.shape
    # x = x[:, -5:-4, :]
    return x.reshape(trials, -1), y

stim_choice = 1

x_train_baseline, y_train_baseline = reshape(spikes_train, behaviour_data_train)
x_test_baseline, y_test_baseline = reshape(spikes_test, behaviour_data_test)

In [ ]:
# train logistic regression model
log_reg = LogisticRegression(penalty='l1', C=1, solver='liblinear', verbose=1)
# log_reg = SVC()
log_reg.fit(x_train_baseline, y_train_baseline[:, stim_choice])
# test accuracy
accuracy_train = accuracy_score(y_train_baseline[:, stim_choice], log_reg.predict(x_train_baseline))
accuracy_test = accuracy_score(y_test_baseline[:, stim_choice], log_reg.predict(x_test_baseline))
print('Logistic Regression Accuracy - train: {:.4f}, test: {:.4f}'.format(accuracy_train, accuracy_test))

In [ ]:
import torch.nn as nn
# Define logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(875, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

# Instantiate the model, loss function, and optimizer
model = LogisticRegressionModel()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 2000
train_losses, test_losses = [], []
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train_baseline)

    # Compute the loss
    loss = criterion(outputs, y_train_baseline[:, stim_choice:stim_choice+1])

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Print the loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Test the model
    with torch.no_grad():    
        predicted_prob = model(x_test_baseline)
        test_losses.append(criterion(predicted_prob, y_test_baseline[:, stim_choice:stim_choice+1]).item())
        predicted_class = (predicted_prob >= 0.5).float()
        accuracy = (predicted_class == y_test_baseline[:, stim_choice:stim_choice+1]).float().mean()
print(f'Accuracy: {accuracy.item():.4f}')


In [ ]:
plt.plot(train_losses, label='train')
plt.plot(test_losses, label='test')
plt.legend()

In [ ]:
# log reg coeff
# plot coefficients in a 2d grid
# c = log_reg.coef_.reshape(-1, 25)
# plt.imshow(c)

# plot nn coefficients
c = model.linear.weight.detach().numpy().reshape(-1, 25)
plt.imshow(c)

In [ ]:
# get mean and standard deviation of decoding loss over trials
model.eval()
# print("Total models to study:", len(test_losses[:10] + test_losses[10::20]))
print("Total models to study:", len(test_losses))
test_losses_aggregated = []
z_agg, x_agg = [], []
cov_norm = []
cnn_weights = []
decoding_runs = 50
# for epoch, _ in test_losses[:10] + test_losses[10::20]:
for epoch, _ in test_losses:
    # print(epoch)
    # if epoch != 331:
    #     continue
    model.load_model(str(epoch))
    cur_epoch_loss = []
    cur_epoch_z, cur_epoch_x = [], []
    with torch.no_grad():
        cov_norm_current = []
        for _ in range(decoding_runs):
            test_loss = []
            cur_run_z, cur_run_x = [], []
            for _, (behavior_batch, spikes_batch) in enumerate(test_loader):
                y_recon, (mu, A), (z, x), behavior_batch_pred = model(spikes_batch)
                behavior_loss = model.behavior_decoder.loss(behavior_batch_pred, behavior_batch, z, reduction='none')                
                test_loss.append(behavior_loss)
                cur_run_z.append(z[:, :, 0])
                cur_run_x.append(x[:, :, 0])
                cov_norm_current.append(np.linalg.norm(A.numpy()))
            # stack horizontally and take mean across samples
            test_loss = torch.mean(torch.cat(test_loss, dim=0), dim=0).item()
            cur_epoch_loss.append(test_loss)
            # stack z and x
            z_stacked, x_stacked = torch.cat(cur_run_z, dim=0), torch.cat(cur_run_x, dim=0)            
            cur_epoch_z.append(z_stacked)
            cur_epoch_x.append(x_stacked)
        
        test_losses_aggregated.append((epoch, np.mean(cur_epoch_loss), np.std(cur_epoch_loss)))
        # compute std for z and x
        z_std, x_std = torch.std(torch.stack(cur_epoch_z), dim=0), torch.std(torch.stack(cur_epoch_x), dim=0)        
        
        z_agg.append((epoch, z_std.mean().item()))
        x_agg.append((epoch, x_std.mean().item()))
        # cov norm
        cov_norm.append((epoch, np.mean(cov_norm_current)))
        # cnn weights
        cnn_weights.append((epoch, deepcopy(model.behavior_decoder.conv_stim)))        
# # plot histogram
# plt.hist(test_loss.numpy(), range=(0, 7), bins=50, density=True)
# plt.xlabel('Cross entropy loss')
# plt.ylabel('Frequency')
# plt.title('Test decoding loss')


In [ ]:
# plot mean and std of test loss
test_losses_plot = np.array(test_losses_aggregated)
# plt.errorbar(test_losses_plot[:, 0], test_losses_plot[:, 1], yerr=test_losses_plot[:, 2])
# print(test_losses_plot)
plt.plot(test_losses_plot[:, 0], test_losses_plot[:, 1])
plt.xlabel('Epoch')
plt.ylabel('Cross entropy loss')
plt.title('Test decoding loss mean and std over 200 runs at various epochs')

In [ ]:
# plot x and z
print(z_agg)
z_agg_plot = np.array(z_agg)
x_agg_plot = np.array(x_agg)
plt.plot(z_agg_plot[:, 0], z_agg_plot[:, 1], label='z')
plt.plot(x_agg_plot[:, 0], x_agg_plot[:, 1], label='x')
plt.xlabel('Epoch')
plt.ylabel('Standard deviation')
plt.title('Standard deviation of z and x over 200 runs')
plt.legend()

In [ ]:
# plot norm of covariance
cov_norm_plot = np.array(cov_norm)
plt.plot(cov_norm_plot[:, 0], cov_norm_plot[:, 1])
plt.xlabel('Epoch')
plt.ylabel('Norm of covariance')

In [ ]:
# examine particular examples
# get mean and standard deviation of decoding loss over trials
model.eval()
print("Total models to study:", len(test_losses))
test_losses_aggregated = []
decoding_runs = 100
for epoch, _ in test_losses[:10] + test_losses[10::20]:
    # print(epoch)
    model.load_model(str(epoch))
    cur_epoch_loss = []    
    with torch.no_grad():        
        for _ in range(decoding_runs):
            test_loss = []
            cur_run_z, cur_run_x = [], []
            for _, (behavior_batch, spikes_batch) in enumerate(test_loader):
                y_recon, (mu, A), (z, x), behavior_batch_pred = model(spikes_batch)
                behavior_loss = model.behavior_decoder.loss(behavior_batch_pred, behavior_batch, z, reduction='none')                
                test_loss.append(behavior_loss)                
            # stack horizontally and take mean across samples
            test_loss = torch.cat(test_loss, dim=0)
            cur_epoch_loss.append(test_loss)
        # stack the losses across decoding runs
        cur_epoch_loss = torch.stack(cur_epoch_loss, dim=0)
        # compute mean and std of epoch loss across decoding runs
        m, s = torch.mean(cur_epoch_loss, dim=0).numpy(), torch.std(cur_epoch_loss, dim=0).numpy()        
        test_losses_aggregated.append((epoch, m, s))


In [ ]:
for idx in range(15):
    epochs = [x[0] for x in test_losses_aggregated]
    losses = [x[2][idx] for x in test_losses_aggregated]
    plt.plot(epochs, losses)
plt.xlabel('Epoch')
plt.ylabel('Cross entropy loss')

    

In [ ]:
# plot norm of change in CNN weights over time
base_cnn = cnn_weights[0][1]
layers_to_study = [(x, base_cnn[x].__module__.split('.')[-1]) for x in range(len(base_cnn)) if isinstance(base_cnn[x], torch.nn.Conv1d) or isinstance(base_cnn[x], torch.nn.BatchNorm1d)]
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
for layer_idx, name in layers_to_study:    
    change_in_norms = []
    for i, (epoch, cur_cnn) in enumerate(cnn_weights):
        if i == 0:
            continue
        cur_weight = cur_cnn[layer_idx].weight.detach().numpy()
        prev_epoch_weights = cnn_weights[i-1][1][layer_idx].weight.detach().numpy()        
        change_in_norm = np.linalg.norm(cur_weight - prev_epoch_weights)        
        # normalize by total number of weights        
        change_in_norm /= cur_weight.size
        change_in_norms.append((epoch, change_in_norm))
    change_in_norms = np.array(change_in_norms)    
    ax1.plot(change_in_norms[:, 0], change_in_norms[:, 1], label=name)

# plot test_losses_og 
decoding_loss_only = np.array([(x[0], x[1][1]) for x in test_losses_og])
ax2.plot(decoding_loss_only[:, 0], decoding_loss_only[:, 1], '-.', 'b')
plt.xlabel('Epoch')
ax1.set_ylabel('Norm of change in CNN weights')
ax2.set_ylabel('Test loss')
plt.title('Norm of change in CNN weights over time')
ax1.legend()
plt.show()